## Import

In [ ]:
# basic libray
import pickle
import numpy as np
import pandas as pd
import warnings; warnings.filterwarnings("ignore")

# surprise Library import
from surprise import Dataset
from surprise import Reader
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor
from surprise import KNNBasic,KNNBaseline, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

# model save
import joblib

## Read data

In [ ]:
data_path = '../data/'

data = pickle.load(open(data_path+'user-item matrix', 'rb'))

## Cross validation

In [ ]:
benchmark = []
# 모든 알고리즘을 literate화 시켜서 반복문을 실행시킨다.
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(),
                  KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    
    # 교차검증을 수행하는 단계.
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # 결과 저장과 알고리즘 이름 추가.
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

* 각 모델별 교차검증 점수에 따르면 BaselineOnly 모델의 test_rmse와 fit_timed이 압도적이므로 최종 모델로 선택한다.
* 하지만 주어진 데이터셋을 넘어 대용량 데이터셋 이용을 위해 메모리 절약 효과가 있는 SVD(잠재요인 추출) 모델 또한 선택한다.

## Hyperparameters tunning

In [ ]:
# 이제 trainset과 testset 전체를 사용하여 학습하기 위해 trainset을 전체로 변환
trainset = data.build_full_trainset()

In [ ]:
# SVD 모델에 대해 GridSearch를 사용한 교차검증 rmse를 사용해 하이퍼파라미터 선정
# n_epochs: SGD 수행 시 반복 횟수, n_factors: 잠재 요인 크기
param_grid = {
    'n_epochs': [20, 40, 60], 
    'n_factors': [50, 100, 200]
}

# GridSearchCV
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3) 
gs.fit(data)

# 최적 하이퍼 파라미터 및 그 때의 최고 성능
print(gs.best_params['rmse'])
print(gs.best_score['rmse'])

In [ ]:
# BaselinOnly 모델과 최적 하아퍼파라미터를 적용한 SVD 모델 학습
base = BaselineOnly(); svd = SVD(**gs.best_params['rmse'])
base.fit(trainset); svd.fit(trainset)

## Save model

In [ ]:
model_path = '../data/model'
joblib.dump(base, data_path + 'base.pkl')
joblib.dump(svd, data_path + 'svd.pkl')